<a href="https://colab.research.google.com/github/AKASH4671/Sentiment-Analysis-on-Financial-News-and-Its-Impact-on-Stock-Prices/blob/main/01_1_investing_com_apple_news_batch_genarator_and_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from datetime import datetime

batch_scripts = []
base_code_template = '''import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time

options = uc.ChromeOptions()
# options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

articles_data = []

try:
    driver = uc.Chrome(options=options)
    driver.set_page_load_timeout(180)

    for page_num in range({start_page}, {end_page}+1):
        url = f"https://www.investing.com/equities/apple-computer-inc-news/{{{{page_num}}}}"
        print(f" Scraping page {{{{page_num}}}}...")

        try:
            driver.get(url)
            time.sleep(5)
        except TimeoutException:
            print(f" Timeout on page {{{{page_num}}}}, skipping...")
            continue

        articles = driver.find_elements(By.CSS_SELECTOR, "div.block.w-full.sm\\\\:flex-1")
        print(f" Found {{{{len(articles)}}}} articles on page {{{{page_num}}}}")

        if len(articles) == 0:
            print(" No more articles. Stopping.")
            break

        for article in articles:
            try:
                title_elem = article.find_element(By.CSS_SELECTOR, "a[data-test='article-title-link']")
                title = title_elem.text.strip()
                link = title_elem.get_attribute("href")

                try:
                    summary_elem = article.find_element(By.CSS_SELECTOR, "p[data-test='article-description']")
                    summary = summary_elem.text.strip()
                except:
                    summary = ""

                try:
                    date_elem = article.find_element(By.CSS_SELECTOR, "time[data-test='article-publish-date']")
                    date = date_elem.get_attribute("datetime")
                except:
                    date = ""

                articles_data.append({{
                    "date": date,
                    "headline": title,
                    "summary": summary,
                    "url": link,
                    "source": "Investing.com"
                }})

            except Exception as e:
                print(" Error reading article:", e)
                continue

    driver.quit()

except Exception as e:
    print(" Critical Error:", e)
    try:
        driver.quit()
    except:
        pass

df = pd.DataFrame(articles_data)
df.to_csv("investing_apple_news_batch_{batch_num}.csv", index=False)
print(f" Scraped {{{{len(df)}}}} articles and saved to 'investing_apple_news_batch_{batch_num}.csv'")
'''

#  Generate 7 scripts for batches
for i in range(7):
    script_code = base_code_template.format(
        start_page=i * 50 + 1,
        end_page=(i + 1) * 50,
        batch_num=i + 1
    )
    filename = f"batch_{i+1}.py"
    with open(filename, "w", encoding="utf-8") as f:
        f.write(script_code)
    print(f" Created {filename}")

 Created batch_1.py
 Created batch_2.py
 Created batch_3.py
 Created batch_4.py
 Created batch_5.py
 Created batch_6.py
 Created batch_7.py
